<a href="https://colab.research.google.com/github/vasid99/cs6910-dl/blob/main/Assignment02/Assignment02_part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Definitions

In [2]:
!pip install -q wandb

     |████████████████████████████████| 2.1MB 8.2MB/s 
     |████████████████████████████████| 133kB 53.4MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 163kB 36.6MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 


In [3]:
import numpy as np
import json
import glob

import tensorflow as tf
ks = tf.keras

import wandb
from wandb.keras import WandbCallback as WandbCallback

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Specifying the architecture

In [5]:
ks_app_CNN = ks.applications.InceptionV3
ks_app_cnn = ks.applications.inception_v3

# Reading iNaturalist Data

In [7]:
path_train = "/content/drive/MyDrive/Sem 8/DL/inaturalist_12K/train"
path_test  = "/content/drive/MyDrive/Sem 8/DL/inaturalist_12K/val"

batch_size = 32
img_size = (800, 800)
img_shape = img_size + (3,)

train_ds = ks.preprocessing.image_dataset_from_directory(path_train,
                                                         label_mode='categorical',
                                                         validation_split = 0.1,
                                                         subset="training",
                                                         seed=123,
                                                         image_size=img_size,
                                                         batch_size=batch_size)

val_ds = ks.preprocessing.image_dataset_from_directory(path_train,
                                                       label_mode='categorical',
                                                       validation_split = 0.1,
                                                       subset="validation",
                                                       seed=123,
                                                       image_size=img_size,
                                                       batch_size=batch_size)

test_ds = ks.preprocessing.image_dataset_from_directory(path_test,
                                                        label_mode='categorical',
                                                        image_size=img_size,
                                                        batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


Found 10026 files belonging to 10 classes.
Using 9024 files for training.
Found 10026 files belonging to 10 classes.
Using 1002 files for validation.
Found 2000 files belonging to 10 classes.


# Creating the model

In [12]:
# Defining data_augmentation function

'''
data_aug = ks.Sequential([
    ks.layers.experimental.preprocessing.RandomFlip('horizontal'),
    ks.layers.experimental.preprocessing.RandomRotation(0.2)
])

# Creating the cnn
cnn_model = ks_app_CNN(input_shape=img_shape,
                      include_top=False,
                      weights='imagenet',
                      pooling='max')

'''
data_aug = ks.Sequential([
    ks.layers.experimental.preprocessing.RandomFlip('horizontal'),
    ks.layers.experimental.preprocessing.RandomRotation(0.2),
    ks.layers.experimental.preprocessing.Resizing(299, 299, interpolation='gaussian')
])

# Creating the cnn
cnn_model = ks_app_CNN(include_top=True,
                      weights='imagenet')


cnn_model.trainable = False

# Creating the model
inputs = ks.Input(shape=img_shape)
outputs = data_aug(inputs)
outputs = ks_app_cnn.preprocess_input(outputs)
outputs = cnn_model(outputs, training=False)
#outputs = tf.keras.layers.Dropout(0.2)(outputs)
outputs = ks.layers.Dense(num_classes, activation='softmax')(outputs)

model = ks.Model(inputs, outputs)
model.compile(optimizer=ks.optimizers.Adam(lr=1e-4),
              loss=ks.losses.categorical_crossentropy,
              metrics=['accuracy'])

96116736/96112376 [==============================] - 1s 0us/step


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 800, 800, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 800, 800, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 800, 800, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 800, 800, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
____________________________________________

In [13]:
epochs = 10
history = model.fit(train_ds,
                   epochs=epochs,
                   validation_data=val_ds)

Epoch 1/10
282/282 [==============================] - 3183s 11s/step - loss: 2.3011 - accuracy: 0.1020 - val_loss: 2.2906 - val_accuracy: 0.2255
Epoch 2/10
282/282 [==============================] - 202s 702ms/step - loss: 2.2879 - accuracy: 0.2829 - val_loss: 2.2772 - val_accuracy: 0.4002
Epoch 3/10
282/282 [==============================] - 201s 701ms/step - loss: 2.2754 - accuracy: 0.4190 - val_loss: 2.2639 - val_accuracy: 0.4850
Epoch 4/10
282/282 [==============================] - 201s 702ms/step - loss: 2.2628 - accuracy: 0.5111 - val_loss: 2.2506 - val_accuracy: 0.5599
Epoch 5/10
282/282 [==============================] - 202s 702ms/step - loss: 2.2504 - accuracy: 0.5851 - val_loss: 2.2374 - val_accuracy: 0.6447
Epoch 6/10
282/282 [==============================] - 202s 704ms/step - loss: 2.2377 - accuracy: 0.6332 - val_loss: 2.2243 - val_accuracy: 0.6786
Epoch 7/10
282/282 [==============================] - 202s 703ms/step - loss: 2.2259 - accuracy: 0.6656 - val_loss: 2.2113 - 

In [1]:
model.summary()

NameError: ignored